In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from torch import optim
from sklearn import preprocessing
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import median_absolute_error

In [18]:
rawdat = pd.read_csv('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\rawdat.csv', sep = ';')

In [19]:
rawdat = rawdat[['HomeTeam','AwayTeam','FTHG','FTAG','FTR','HTR']]